In [1]:
%load_ext autoreload
%autoreload 2

The goal of this notebook is to automate the creation of MMS for the multicompartment model, by using Miro's ulfy to translate between ufl and sympy expression.

In [2]:
from dolfin import *
from ulfy import Expression
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np

import pygmsh
import meshio
import gmsh
from pathlib import Path
from datetime import datetime
from multirat.meshprocessing import meshio2xdmf, xdmf2fenics

In [3]:
def is2dimensional(mesh):
    return all(mesh.points[:, 2] == 0.0)

def meshio2fenics(mesh):
    if is2dimensional(mesh):
        dim = 2
    else:
        dim = 3
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    p = Path(f"/tmp/mesh-{timestamp}")
    p.mkdir()
    meshio2xdmf(mesh, p, dim)
    return xdmf2fenics(p)

def create_gmsh_circle(h):
    if not gmsh.is_initialized():
        gmsh.initialize()
    gmsh.model.add("Unit Disk")
    gmsh.model.occ.add_disk(0, 0, 0, 1, 1)
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    
    gmsh.model.occ.synchronize()
    gmsh.model.mesh.generate(2)

    gmsh.write("mesh.msh")
    gmsh.clear()
    gmsh.finalize()

Miros Code

In [4]:
# With (0, 1)^2 and boundaries
#    4
#  1   2
#    3
#
# we want to solve -div kappa grad u + u = f in (0, 1)^2
#                                      u = g_D somewhere on boundary
#               -kappa * du/dn - alpha*u = g_R somewhere on boundary
#                           -kappa*du/dn = g_N somewhere on boundary
import numpy as np
from dolfin import *


# Define unit normaal vector of boudnaries.
normals = {1: Constant((-1, 0)), 2: Constant((1, 0)), 3: Constant((0, -1)), 4: Constant((0, 1))}


# Define various subdomains.
subdomains = {1: CompiledSubDomain('near(x[0], 0)'),
              2: CompiledSubDomain('near(x[0], 1)'),
              3: CompiledSubDomain('near(x[1], 0)'),
              4: CompiledSubDomain('near(x[1], 1)')}


#
def solve_poisson(kappa, alpha, boundaries, f_true, Ddata, Rdata, Ndata):
    '''Solve the problem...'''
    # Verify that all boundaries are provided with some data.
    assert (Ddata.keys() | Rdata.keys() | Ndata.keys()) == set((1, 2, 3, 4))\
    
    # Not sure.
    tags = tuple(map(set, (Ddata.keys(), Rdata.keys(), Ndata.keys())))
    assert all(not this & that for i, this in enumerate(tags) for that in tags[i+1:])

    mesh = boundaries.mesh()
    V = FunctionSpace(mesh, 'CG', 2)
    u, v = TrialFunction(V), TestFunction(V)

    ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

    a = inner(kappa*grad(u), grad(v))*dx + inner(u, v)*dx
    L = inner(f_true, v)*dx    
    # Robin to system
    for tag in Rdata:
        a += inner(alpha*u, v)*ds(tag)
        L -= inner(Rdata[tag], v)*ds(tag)
    # Neumann
    n = FacetNormal(mesh)
    L += -sum(inner(dot(Ndata[tag], n), v)*ds(tag) for tag in Ndata)

    bcs = [DirichletBC(V, Ddata[tag], boundaries, tag) for tag in Ddata]

    uh = Function(V)
    solve(a == L, uh, bcs=bcs)

    return uh

# --------------------------------------------------------------------

In [6]:
from tabulate import tabulate
import sympy as sp
import ulfy


# Define MMS Place holders
x, y = SpatialCoordinate(UnitSquareMesh(1, 1))

kappa = 1 + x + y
u = sin(pi*(x+y))
sigma = -kappa*grad(u)
f = div(sigma) + u

alpha = Constant(1)  # Place holder

robin_piece = lambda n: dot(sigma, n) - alpha*u

alpha0 = sp.Symbol('alpha')

# Convert for dolfin: when converting alpha to sympy alpha will become
# alpha0 and in the Expression we can then set the value without
# generating new code
expr = lambda v: ulfy.Expression(v, subs={alpha: alpha0}, degree=4, alpha=alpha_value)  # Here we set it

robin_expr = {tag: expr(robin_piece(n)) for tag, n in normals.items()}

# -----

u_true, f_true, sigma_true, kappa = map(expr, (u, f, sigma, kappa))
alpha = Constant(alpha_value)

Ddata = {tag: u_true for tag in (1, )}
Ndata = {tag: sigma_true for tag in (2, )}
Rdata = {tag: robin_expr[tag] for tag in (3, 4)}

history = []
for n in (8, 16, 32, 64, 128):
    # Finer for bulk
    mesh = UnitSquareMesh(n, n)
    boundaries = MeshFunction('size_t', mesh, 1, 0)
    [subd.mark(boundaries, tag) for tag, subd in subdomains.items()]

    uh = solve_poisson(kappa, alpha, boundaries, f_true, Ddata, Rdata, Ndata)

    h = mesh.hmin()
    eu = errornorm(u_true, uh, 'H1')

    history.append((h, eu))

    print('-'*72)
    print(tabulate(history, headers=('h', '|eu|_1')))
    print('-'*72, '\n')

NameError: name 'alpha_value' is not defined

In [7]:
tags

({1}, {3, 4}, {2})

In [10]:
for ex in (not this & that for i, this in enumerate(tags) for that in tags[i+1:]):
    print(ex)

True
True
True


In [36]:
(not True) & False

False

In [31]:
# Not sure.
tags = tuple(map(set, (Ddata.keys(), Rdata.keys(), Ndata.keys())))

# I think it is a check for whether any tags are repeaded?
def valid_data():
    N = len(tags)
    these = [False] * (N-1)
    for i, this in enumerate(tags):
        for that in tags[i+1:]:
            these[i] = (not this & that)
    return these
        
# Checks for empty unions between different sets of tags.
assert all(not this & that for i, this in enumerate(tags) for that in tags[i+1:])

In [30]:
valid_data()

[True, True]

In [5]:
# The API also allows to set a global mesh size callback, which is called each
# time the mesh size is queried
def meshSizeCallback(dim, tag, x, y, z, lc=1):
    r = x**2 + y**2
    if r < 0.7:
        return 1.0
    return 0.05

In [11]:
if not gmsh.is_initialized():
    gmsh.initialize()
gmsh.model.add("Unit Disk")
circle = gmsh.model.occ.add_circle(0, 0, 0, 1, 1)
loop = gmsh.model.occ.add_curve_loop([circle])
domain = gmsh.model.occ.add_plane_surface([loop])
gmsh.model.occ.synchronize()

# distance = gmsh.model.mesh.field.add("Distance")
# gmsh.model.mesh.field.setNumbers(distance, "PointsList", [1])
# gmsh.model.mesh.field.setNumbers(distance, "CurvesList", [circle])
# gmsh.model.occ.synchronize()

# threshold = gmsh.model.mesh.field.add("Threshold")
# gmsh.model.mesh.field.setNumber(threshold, "IField", distance)
# gmsh.model.mesh.field.setNumber(threshold, "LcMin", 0.05)
# gmsh.model.mesh.field.setNumber(threshold, "LcMax", 1.0)
# gmsh.model.mesh.field.setNumber(threshold, "DistMin", 0.0)
# gmsh.model.mesh.field.setNumber(threshold, "DistMax", 1.0)
# gmsh.model.mesh.field.setAsBackgroundMesh(threshold)

# minimum = gmsh.model.mesh.field.add("Min")
# gmsh.model.mesh.field.setNumbers(minimum, "FieldsList", [threshold])
# gmsh.model.mesh.field.setAsBackgroundMesh(minimum)

# The API also allows to set a global mesh size callback, which is called each
# time the mesh size is queried
def meshSizeCallback(dim, tag, x, y, z, lc=1):
    r = x**2 + y**2
    if r < 0.5:
        return 1.0
    return 0.05

gmsh.model.mesh.setSizeCallback(meshSizeCallback)
gmsh.model.occ.synchronize()
gmsh.model.mesh.setSizeCallback(meshSizeCallback)
gmsh.model.mesh.generate(2)

gmsh.write("mesh.msh")
gmsh.clear()
gmsh.finalize()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Circle)
Info    : Done meshing 1D (Wall 0.000587702s, CPU 0.000457s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0796133s, CPU 0.079805s)
Info    : 1550 nodes 3099 elements
Info    : Writing 'mesh.msh'...
Info    : Done writing 'mesh.msh'
Info    : Clearing all models and views...
Info    : Done clearing all models and views


In [12]:
mesh = meshio.read("mesh.msh")
mesh, _, _ = meshio2fenics(mesh)
mesh

In [ ]:
def create_circlemesh(h):
    create_gmsh_circle(h)

    mesh = meshio.read("mesh.msh")
    mesh, _, _ = meshio2fenics(mesh)
    return mesh

mesh = create_circlemesh(0.5)
mesh

In [ ]:
V = FunctionSpace(mesh, "CG", 2)
A = 1.0
p_expr = Expression("A * (x[0]*x[0] + x[1]*x[1])", degree=2, A=A)
p = TrialFunction(V)
q = TestFunction(V)

F = inner(grad(p), grad(q)) * dx
F -= -4*A * q * dx  # TODO: Replace by autogenerated expression.

a = lhs(F)
L = rhs(F)

bcs=[DirichletBC(V, Constant(A), "on_boundary")]
p = Function(V)
solve(a == L, p, bcs=bcs)

e = Function(V)
e.assign(p - project(p_expr, V))

c = plot(e)
plt.colorbar(c)
plt.show()

In [35]:
# mesh = create_circlemesh(h=0.05)
# V = FunctionSpace(mesh, "CG", 2)
# A = 1.0
# p_expr = Expression("A * (x[0]*x[0] + x[1]*x[1])", degree=2, A=A)
# p = TrialFunction(V)
# q = TestFunction(V)

# F = inner(grad(p), grad(q)) * dx
# F -= -4*A * q * dx  # TODO: Replace by autogenerated expression.

# a = lhs(F)
# L = rhs(F)

# bcs=[DirichletBC(V, Constant(A), "on_boundary")]

# p = Function(V)
# solve(a == L, p, bcs=bcs)
# c = plot((p - p_expr))
# plt.colorbar(c)
# plt.show()

0.0006215393905389943

In [32]:
norm(e, "L2")

0.0007455075674200319

In [30]:
e = Function(V)
e.assign(p - interpolate(p_expr, V))
norm(e.vector(), "l2")

0.03263991331739253

In [ ]:
# a = 1.0
# x, y, t = sp.symbols("x y t")
# p = a * (x**2 + y**2)

# def robin_boundary(p, K, a):
#     x = SpatialCoordinate(mesh)
#     return p - K / a * inner(grad(p), x)

# from dolfin import Expression as dolfExpr

# from collections import defaultdict

# x, y, t = sp.symbols("x y t")
# p_sp = a * (x**2 + y**2)

# K = 1.0
# G = defaultdict(lambda: 1.0)

# # gR = robin_boundary(pxpr, 0.1, 0.2)